<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-1-Optimizaci-n-Matem-tica/blob/main/P1b%20buena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gurobipy
import numpy as np #importamos la libreria de numpy para manejar matrices.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 63.9 MB/s eta 0:00:00


In [4]:
#definamos la matriz D de la siguiente manera:

D = np.array([[4,6,5,9,17,4],[8,12,3,7,11,5],[6,8,2,10,15,7],[5,9,4,8,13,6],
 [7,11,3,12,8,10],[10,5,6,4,9,7]])
print(D)
np.shape(D) #para comprobar la dimensión de la matriz


print(D[0,0])

[[ 4  6  5  9 17  4]
 [ 8 12  3  7 11  5]
 [ 6  8  2 10 15  7]
 [ 5  9  4  8 13  6]
 [ 7 11  3 12  8 10]
 [10  5  6  4  9  7]]
4


In [5]:
#primero voy a sacar los esfuerzos que requiere cada robot para mover cada objeto:
R1 = D[:,0]
R2 = D[:,1]
R3 = D[:,2]
R4 = D[:,3]
R5 = D[:,4]
R6 = D[:,5]


In [6]:
#ahora usaremos gurobi para optimizar el problema:
from gurobipy import*

D = D = np.array([[4,6,5,9,17,4],[8,12,3,7,11,5],[6,8,2,10,15,7],[5,9,4,8,13,6],
 [7,11,3,12,8,10],[10,5,6,4,9,7]])


n = D.shape[0] # numeros de elemento de elementos de objetos que coincide con el numero de robots

model = Model("Robots_Trabajadores")
model.setParam('OutputFlag',0)

#definamos las varaibles del modelos

#como vamos a trabajar con variables binarias, se tiene que x_{i,j} toma valores {0,1} segun las condiciones colocadas en el informe.

x = model.addVars(n,n, vtype=GRB.CONTINUOUS,lb=0, ub=1, name = 'x')

tanh_D= np.tanh(D-5)

model.update()
#Definimos la función objetivo:

model.setObjective(quicksum(x[i,j]*(50000*tanh_D[i,j] + 50000) for i in range(n) for j in range(n)), GRB.MINIMIZE)


#restricciones:

for i in range(n):
  model.addConstr(quicksum(x[i,j] for j in range(n)) == 1, name = f'restriccion_robot_{i}')

for j in range(n):
  model.addConstr(quicksum(x[i,j] for i in range(n)) == 1, name = f'restriccion_robot_{j}')

model.optimize()

costo_final = 0
for i in range(n):
    for j in range(n):
        if x[i, j].X > 0.5:
            costo = 50000 * tanh_D[i, j] + 50000
            costo_final += costo

            print(f"Robot {i+1} asignado a objeto {j+1} con costo {costo:.2f}")

print(f"\nCosto total de la asignación: {costo_final:.2f}")

Valores = np.array([[11920.29, 50000, 247.26, 99966.46, 99752.74, 11920.29]])

Restricted license - for non-production use only - expires 2026-11-23
Robot 1 asignado a objeto 1 con costo 11920.29
Robot 2 asignado a objeto 6 con costo 50000.00
Robot 3 asignado a objeto 3 con costo 247.26
Robot 4 asignado a objeto 2 con costo 99966.46
Robot 5 asignado a objeto 5 con costo 99752.74
Robot 6 asignado a objeto 4 con costo 11920.29

Costo total de la asignación: 273807.05


In [11]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def costo(d):
    return 50000 * np.tanh(d - 5) + 50000

# Matrices de esfuerzo para las cintas
D1 = np.array([  #esfuerzo para llegar a las cintas por robot
    [1, 1, 5, 1, 1, 2],
    [2, 1, 5, 1, 2, 1]
])

D2 = np.array([  #esfuerzo para llegar a las cajas, dado que usó la cinta
    [1, 1, 4, 1, 2, 1],
    [1, 2, 4, 1, 1, 1]
])
# Tarifa fija por robot que usó la cinta
tarifa = 10000
# Número de cintas
k = D1.shape[0]

# Crear el modelo
m1 = gp.Model("AsignacionRobotsCintas")

# Variables binarias
#x define que algún robot lleve el objeto i a la caja j
x1 = m1.addVars(n, n, vtype=GRB.CONTINUOUS, name="x_directo")
#y define el robot que lleva el objeto, qué cinta usa y a qué caja lo lleva
y = m1.addVars(n, n, vtype=GRB.CONTINUOUS, name="y_cinta")
# Función objetivo

#calcula los costos en función de la variable X para el camino directo. Va casilla por casilla evaluando
CostosD = gp.quicksum(x1[i,j] * costo(D[i][j]) for i in range(n) for j in range(n))
#calcula el costo de por cinta, sumando el costo del viaje inicial con el final, más la tarifa
costosC = gp.quicksum(y[i,j] * (min(costo(D1[0][i]) + costo(D2[0][j]),costo(D1[1][i]) + costo(D2[1][j])) + tarifa) for i in range(n) for j in range(n))
#al ser continua, considera el caso en que tome la cinta y también camine, pero en el sentido práctico esa opción raramente se presentaría
m1.setObjective(CostosD + costosC, GRB.MINIMIZE)

# restricciones
# a cada robot asigna exactamente un objeto
for i in range(n):
    m1.addConstr(gp.quicksum(x1[i,j] + y[i,j] for j in range(n)) == 1)


# Cada caja recibe exactamente un objeto (puede ser por la cinta o caminando)
for j in range(n):
    m1.addConstr(gp.quicksum(x1[i,j] + y[i,j] for i in range(n)) == 1)

# Optimizar
m1.optimize()

# Mostrar resultados
if m1.status == GRB.OPTIMAL:
    for j in range(n):
        for i in range(n):
          # Asignaciones directas (sin cintas)
            if x1[i,j].X > 0.5:
                print(f"Robot {i+1} tomó el objeto {i+1} y lo llevó directamente a la caja {j+1}. Costo: {costo(D[i,j]):.2f}")
            if y[i,j].X > 0.5:
                costo_cinta1 = costo(D1[0,i]) + costo(D2[0,j]) + tarifa
                costo_cinta2 = costo(D1[1,i]) + costo(D2[1,j]) + tarifa
                mejor_cinta = 1 if costo_cinta1 < costo_cinta2 else 2
                print(f"Robot {i+1} tomó el objeto {i+1} y lo llevó a la caja {j+1} en la cinta: {mejor_cinta}, Costo: {min(costo_cinta1, costo_cinta2):.2f})")
    print(f"\nCosto total mínimo: {m1.ObjVal:.2f}")
else:
    print("No se encontró solución óptima.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 72 columns and 144 nonzeros
Model fingerprint: 0x4891d61b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 36 columns
Presolve time: 0.01s
Presolved: 12 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4045714e+04   6.000000e+00   0.000000e+00      0s
       4    5.0582612e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.058261245e+04
Robot 5 tomó el objeto 5 y lo llevó a la caja 1 en la cinta: 1, Costo: 10067.07)
Robot 2 tomó el objeto 2 y lo llevó a la caj

In [8]:
#P1.C


costo_extra = 2000  # Nuevo costo adicional
max_cintas = 5       #Máximo de robots que pueden usar las cintas

# Modelo
m2 = gp.Model("AsignacionRobotsCintas")

# Variables binarias
#x define que algún robot lleve el objeto i a la caja j
x = m2.addVars(n, n, vtype=GRB.CONTINUOUS, name="x_directo")
#y define el robot que lleva el objeto, qué cinta usa y a qué caja lo lleva
y = m2.addVars(n, n, vtype=GRB.CONTINUOUS, name="y_cinta")
# Función objetivo

#calcula los costos en función de la variable X para el camino directo. Va casilla por casilla evaluando
CostosD = gp.quicksum(x[i,j] * costo(D[i][j]) for i in range(n) for j in range(n))
#calcula el costo de por cinta, sumando el costo del viaje inicial con el final, más la tarifa
costosC = gp.quicksum(y[i,j] * (min(costo(D1[0][i]) + costo(D2[0][j]),costo(D1[0][i]) + costo(D2[0][j]) ) + tarifa + costo_extra) for i in range(n) for j in range(n))
#al ser continua, considera el caso en que tome la cinta y también camine, pero en el sentido práctico esa opción raramente se presentaría
m2.setObjective(CostosD + costosC, GRB.MINIMIZE)

# restricciones
# a cada robot asigna exactamente un objeto
for i in range(n):
    m2.addConstr(gp.quicksum(x[i,j] + y[i,j] for j in range(n)) == 1)


# Cada caja recibe exactamente un objeto (puede ser por la cinta o caminando)
for j in range(n):
    m2.addConstr(gp.quicksum(x[i,j] + y[i,j] for i in range(n)) == 1)

m2.addConstr(quicksum(y[i,j] for i in range(n) for j in range(n)) <= max_cintas) #la cantidad de robots que usan cinta debe ser menor o igual a 5

# Optimizar
m2.optimize()

# Mostrar resultados
if m2.status == GRB.OPTIMAL:
    # Asignaciones directas (sin cintas)
    for j in range(n):
        for i in range(n):
          # Asignaciones directas (sin cintas)
            if x[i,j].X > 0.5:
                print(f"Robot {i+1} tomó el objeto {i+1} y lo llevó directamente a la caja {j+1}. Costo: {costo(D[i,j]):.2f}")
            if y[i,j].X > 0.5:
                costo_cinta1 = costo(D1[0,i]) + costo(D2[0,j]) + tarifa + costo_extra
                costo_cinta2 = costo(D1[1,i]) + costo(D2[1,j]) + tarifa + costo_extra
                mejor_cinta = 1 if costo_cinta1 < costo_cinta2 else 2
                print(f"Robot {i+1} tomó el objeto {i+1} y lo llevó a la caja {j+1} en la cinta: {mejor_cinta}, Costo: {min(costo_cinta1, costo_cinta2):.2f})")
    print(f"\nCosto total mínimo: {m2.ObjVal:.2f}")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 13 rows, 72 columns and 180 nonzeros
Model fingerprint: 0xbcc056a5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.01s
Presolved: 13 rows, 72 columns, 180 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9605381e+04   6.000000e+00   0.000000e+00      0s
       6    6.0502784e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.050278410e+04
Robot 5 tomó el objeto 5 y lo llevó a la caja 1 en la cinta: 1, Costo: 12067.07)
Robot 2 tomó el objeto 2 y lo llevó a la caja 2 en la cinta: 1, Costo: 12067.07)
R